In [1]:
import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import schedule
import time as tt

C:\Users\DELL\AppData\Local\Temp\ipykernel_21920\852437405.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def fget_data(currency,num_c):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_from_pos(currency, mt5.TIMEFRAME_M15, 0,num_c))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
def find_status(data,size):
    data['AbsDiff'] = abs(data['Close'] - data['Open'])

# Find the maximum value among the calculated differences
    max_diff = data['AbsDiff'].max()
    if max_diff>size:
        return 1
    else:
        return 0
    
def find_sl(data,tp):
    if tp==-1:
        max_value = data['High'].max()
        return max_value
    else:
        min_value = data['Low'].min()
        return min_value   


In [3]:
begin_max_value=2045.76
begin_min_value=2042.76


In [4]:
def get_first_values():
    global begin_max_value
    global begin_min_value
    nowd=fget_data('GOLD',110)
    nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
    last_row = nowd.iloc[-2]
    target_date = last_row['mtime'].date()
    print('activated')
    # Filter the DataFrame for the specific day
    day_data = nowd[nowd['mtime'].dt.date == target_date]

    # Find the maximum and minimum points for the day
    max_price = day_data['High'].max()
    min_price = day_data['Low'].min()
    begin_max_value = max_price
    begin_min_value = min_price
    
# get_first_values()
def update_min_max():
    global begin_max_value
    global begin_min_value
    nowd=fget_data('GOLD',110)
    nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
    last_row = nowd.iloc[-2]
    minv=begin_min_value
    maxv=begin_max_value
    price=last_row['Close']
    if price>maxv:
        begin_max_value=last_row['High']
        print('updated upside')
    elif price<minv:
        print('updated downside')
        begin_min_value=last_row['Low']
        
    

In [5]:
def agent1(maxdistance,candlesize,df_profit,lotsize,deviat):
    global begin_max_value
    global begin_min_value
    nowd=fget_data('GOLD',110)
    nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
    last_row = nowd.iloc[-2]
    pg=last_row['mtime'].time()

    if datetime.time(3, 0) <= pg <= datetime.time(16, 30):
        
        maxv=begin_max_value
                # print(maxv)
        minv=begin_min_value
        pos_info = mt5.positions_total()
        if pos_info is None:
            pass

        # Check if there are open positions or active orders
        elif pos_info==0:
            price=last_row['Close']
            if price>maxv:
                    if distance_checker(price,maxv,maxdistance)==0:
                        begin_max_value=last_row['High']
                    elif distance_checker(price,maxv,maxdistance)==1:
                        # print('i am here 4')
                        win_data=nowd.tail(6)
                        if find_status(win_data,candlesize):

                            sl_data=nowd.tail(3)
                            sl=int(find_sl(sl_data,1))-1.5
                            tp=price+df_profit
                            begin_max_value=last_row['High']
                            lot=lotsize
                            deviation=deviat
                            request = {"action": mt5.TRADE_ACTION_DEAL,"symbol": 'GOLD',"volume": lot,"type": mt5.ORDER_TYPE_BUY,"price": price,"sl": sl,"tp": tp,"deviation": deviation,"magic": 20022,"comment": "python script open","type_time": mt5.ORDER_TIME_GTC,"type_filling": mt5.ORDER_FILLING_IOC,}
                            mt5.order_send(request)

            elif price<minv:
                if distance_checker(price,minv,maxdistance)==0:
                    begin_min_value=last_row['Low']

                elif distance_checker(price,maxv,maxdistance)==1:
                    # print('i am here 4')
                    win_data=nowd.tail(6)
                    if find_status(win_data,candlesize):

                        sl_data=nowd.tail(3)
                        sl=int(find_sl(sl_data,-1))+1.5
                        tp=price-df_profit
                        begin_min_value=last_row['Low']
                        lot=lotsize
                        deviation=deviat
                        request = {"action": mt5.TRADE_ACTION_DEAL,"symbol": 'GOLD',"volume": lot,"type": mt5.ORDER_TYPE_SELL,"price": price,"sl": sl,"tp": tp,"deviation": deviation,"magic": 20022,"comment": "python script open","type_time": mt5.ORDER_TIME_GTC,"type_filling": mt5.ORDER_FILLING_IOC,}
                        mt5.order_send(request)




In [6]:
def closetrade():
    nowd=fget_data('GOLD',110)
    pos_info = mt5.positions_total()
    if pos_info is None:
        pass
    elif pos_info>1:
        for position in mt5.positions_get():
            for position in mt5.positions_get():
                if position.symbol.startswith('GOLD'):
                    if position.profit>5:
                 # Your existing code
                        order_type = position.type
                        symbol = position.symbol
                        volume = position.volume
                        if order_type == mt5.ORDER_TYPE_BUY:
                            order_type = mt5.ORDER_TYPE_SELL
                            price = mt5.symbol_info_tick(symbol).bid
                        else:
                            order_type = mt5.ORDER_TYPE_BUY
                            price = mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "symbol": symbol,
                            "volume": float(volume),
                            "type": order_type,
                            "position": position.ticket,
                            "price": price,
                            "magic": 20022,
                            "comment": "Close trade",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }

                        mt5.order_send(close_request)

                
                    

    

In [18]:
def liveprof(self):
    nowd=fget_data('GOLD',110)
    pos_info = mt5.positions_total()

    for position in mt5.positions_get():
        
        
        if position.symbol.startswith('GOLD'):
            return position.profit
            
liveprof()

TradePosition(ticket=202772803, time=1707504962, time_msc=1707504962322, time_update=1707504962, time_update_msc=1707504962322, type=0, magic=0, identifier=202772803, reason=0, volume=0.01, price_open=2021.18, sl=0.0, tp=0.0, price_current=2021.19, swap=0.0, profit=0.01, symbol='GOLD', comment='', external_id='')
0.01


In [ ]:
startdate=0
todaydate=datetime.date.today()
mt5.initialize()
ac='305174435'
lastweek=todaydate-datetime.timedelta(days=100)
print(lastweek)
# hd=fget_data('GOLD',110)
h=mt5.history_deals_get(lastweek,datetime.date.today())
print(h)
mt5.last_error()

In [3]:
import datetime
from datetime import timedelta
import MetaTrader5 as mt5

today_date = datetime.datetime.today()
last_week = today_date - timedelta(days=100)

# Convert to Unix timestamps
last_week_timestamp = datetime.datetime.timestamp(last_week)
today_timestamp = datetime.datetime.timestamp(today_date)
print(last_week_timestamp)

history_deals = mt5.history_deals_get(last_week_timestamp, today_timestamp, 'GOLD')


1698782330.851261


In [ ]:
print(history_deals)
print(mt5.last_error()) 

In [ ]:
today_date = datetime.datetime.now()
# last_week = today_date - datetime.timedelta(days=100)
nm=datetime.datetime(2024, 1, 1)

symbol = 'GOLD'



# Try to get history deals
history_deals = mt5.history_deals_get(nm, today_date, symbol=symbol)

# Print history deals
print("History Deals:")
print(history_deals)
print(type(history_deals))



In [4]:
def get_balance():
    hd=fget_data('GOLD',110)
    account_info = mt5.account_info()
    balance = account_info.balance
    return balance



In [5]:
def get_profit():
    nowd=fget_data('GOLD',110)
    pos_info = mt5.positions_total()
    if pos_info is None:
        pass
    elif pos_info>1:
        for position in mt5.positions_get():
            for position in mt5.positions_get():
                if position.symbol.startswith('GOLD'):
                    return position.profit

In [ ]:
columns = ['ticket', 'order', 'time', 'time_msc', 'type', 'entry', 'magic', 'position_id', 'reason', 'volume', 'price', 'commission', 'swap', 'profit', 'fee', 'symbol', 'comment', 'external_id']

# Create a pandas DataFrame
df = pd.DataFrame(history_deals, columns=columns)
df.drop(index=0, inplace=True)
df

In [ ]:
df['time'] = pd.to_datetime(df['time'])


In [6]:
from pymongo import MongoClient
client = MongoClient()

In [7]:
db = client["agent_1"]

In [9]:
import random

# Generate a random 6-digit number
code = random.randint(100000, 999999)



In [10]:
def initialize_agent_data(initial_balance):
    agent_collection = db["agent_client_data"]
    
    if agent_collection.find_one() is None:
        if initial_balance ==0 or initial_balance is None:
            #send message to input balance
            pass
        else:
            agent_activation_date = datetime.datetime.now()
            defaultlotsize=0.01
            defaultdeviation=1
            app_code=random.randint(100000, 999999)
            defaultcomment="agent_1 D.N.G"
            post = {"agent_activation_date": agent_activation_date, "defaultlotsize": defaultlotsize, "defaultdeviation": defaultdeviation, "defaultcomment": defaultcomment,'appcode':app_code}
            agent_collection.insert_one(post)
            parameters_collection = db["agent_parameters"]
            default_maxdistance=1.5
            default_activationtime="03:00"
            default_stoptime="16:30"
            default_profit=5
            default_slwindow=2
            default_candlesize=1.5
            parameter_post = {"appcode":app_code,"default_maxdistance": default_maxdistance, "default_activationtime": default_activationtime, "default_stoptime": default_stoptime, "default_profit": default_profit, "default_slwindow": default_slwindow, "default_candlesize": default_candlesize,'initial_balance':initial_balance}
            parameters_collection.insert_one(parameter_post)
    
                


In [13]:
initialize_agent_data(get_balance())

In [11]:
agent_activation_date=datetime.datetime(2024, 1, 1)#agent_data['agent_activation_date']
todays_date=datetime.datetime.now()
# mt5.history_orders_get(agent_activation_date, todays_date, symbol=symbol)


In [29]:
def generate_profitdata(act_time,todays_date,symbol='GOLD'):
    hd=fget_data('GOLD',110)
    trade_history=mt5.history_deals_get(act_time, todays_date, symbol=symbol)
    if trade_history is None:
        return None
    else:
        columns = ['ticket', 'order', 'time', 'time_msc', 'type', 'entry', 'magic', 'position_id', 'reason', 'volume', 'price', 'commission', 'swap', 'profit', 'fee', 'symbol', 'comment', 'external_id']

        # Create a pandas DataFrame
        df = pd.DataFrame(trade_history, columns=columns)
        return df

In [30]:
def generate_timedata(act_time,todays_date,symbol='GOLD'):
    hd=fget_data('GOLD',110)
    trade_history=mt5.history_orders_get(act_time, todays_date, symbol=symbol)
    if trade_history is None:
        return None
    else:
        data = {
            "ticket": [order.ticket for order in trade_history],
            "time_setup": [order.time_setup for order in trade_history],
            "time_setup_msc": [order.time_setup_msc for order in trade_history],
            "time_done": [order.time_done for order in trade_history],
            "time_done_msc": [order.time_done_msc for order in trade_history],
            "time_expiration": [order.time_expiration for order in trade_history],
            "type": [order.type for order in trade_history],
            "type_time": [order.type_time for order in trade_history],
            "type_filling": [order.type_filling for order in trade_history],
            "state": [order.state for order in trade_history],
            "magic": [order.magic for order in trade_history],
            "position_id": [order.position_id for order in trade_history],
            "position_by_id": [order.position_by_id for order in trade_history],
            "reason": [order.reason for order in trade_history],
            "volume_initial": [order.volume_initial for order in trade_history],
            "volume_current": [order.volume_current for order in trade_history],
            "price_open": [order.price_open for order in trade_history],
            "sl": [order.sl for order in trade_history],
            "tp": [order.tp for order in trade_history],
            "price_current": [order.price_current for order in trade_history],
            "price_stoplimit": [order.price_stoplimit for order in trade_history],
            "symbol": [order.symbol for order in trade_history],
            "comment": [order.comment for order in trade_history],
            "external_id": [order.external_id for order in trade_history],
        }

        # Creating DataFrame
        df = pd.DataFrame(data)
        return df


In [32]:
# 

def make_newdata(df1,df2):
    
    new_df = pd.DataFrame(columns=['open_date', 'close_date', 'profit', 'ticket', 'lot_size','trade_type'])

    rows_to_append = []

    for _, i in df1.iterrows():
        for _, j in df2.iterrows():
            if i['order'] == j['ticket']:
                if i['profit'] != 0.0:
                    new_row = {
                        'open_date': j['time_setup'],
                        'close_date': j['time_done'],
                        'profit': i['profit'],
                        'ticket': i['ticket'],
                        'lot_size': i['volume'],
                        'trade_type': i["type"]
                    }
                    rows_to_append.append(new_row)

    new_df = pd.concat([new_df, pd.DataFrame(rows_to_append)], ignore_index=True)

    return new_df

In [33]:
def finished_data(act_time,todays_date):
    profit_data=generate_profitdata(act_time,todays_date)
    # print(profit_data)
    time_data=generate_timedata(act_time,todays_date)
    # print(time_data)
    if profit_data is None or time_data is None:
        return None
    else:
        final_data=make_newdata(profit_data,time_data)
        return final_data

In [34]:
def updatetrades():
    agent_data=db.agent_client_data.find_one()
    appcode=agent_data['appcode']
    agent_activation_date=datetime.datetime(2024, 1, 1)#agent_data['agent_activation_date']
    todays_date=datetime.datetime.now()
    trade_history=finished_data(agent_activation_date,todays_date)
    # print(trade_history)
    # print(trade_history)    
    if trade_history is None:
        pass
    else:
        

    # Create a pandas DataFrame
        df = trade_history
        df.drop(index=0, inplace=True)
        df['open_date'] = pd.to_datetime(df['open_date'],unit='s')

        trades_collection = db["trades"]
        for index, row in df.iterrows():
            # print(row['ticket'])
            trade_exist_check=trades_collection.find_one({"ticket":row['ticket']})
            # print(trade_exist_check)
            if trade_exist_check is None:
                if row['trade_type']==0:
                    row['trade_type']="Sell"
                elif row['trade_type']==1:
                    row['trade_type']="Buy"
                # print('jjjj')
                # open_time = datetime.datetime.fromtimestamp(row['time'])

                post = {"ticket": row['ticket'], "trade_open":row['open_date'] ,"profit": row['profit'],"lot_size": row['lot_size'],"appcode":appcode,"trade_type":row['trade_type']}
                trades_collection.insert_one(post)

        





In [ ]:
fg=db['trades']
print(fg)
j=fg.find_one({'ticket': '201052422'})
print(j)


In [35]:
updatetrades()

C:\Users\DELL\AppData\Local\Temp\ipykernel_18320\3217958250.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame(rows_to_append)], ignore_index=True)


In [ ]:
# start_date = datetime.datetime(2024, 1, 1)
# end_date = datetime.datetime(2024, 2, 1)  # Note: We're including February 1st to cover the entire month of January

# Construct the query to find documents within the specified date range
# query = {"trade_open": {"$gte": start_date, "$lt": end_date}}

# for i in ttt:
    # print(i)

In [ ]:
ttt=db.trades.find()
df_ttt = pd.DataFrame(ttt)
# print(df_ttt)
highest_profit = df_ttt['profit'].max()
print(highest_profit)


In [ ]:
def get_today_profit():
    today_value=df_ttt[df_ttt['trade_open'].dt.date == datetime.date(2024, 2, 5)]#datetime.date.today()]
    today_profit=today_value['profit'].sum()
    return today_profit


In [ ]:
def get_weekly_data():

    # Get the current week's start and end dates
    today = datetime.date.today()
    start_of_week = today - datetime.timedelta(days=today.weekday())
    end_of_week = start_of_week + datetime.timedelta(days=6)

    # Filter the DataFrame to include values belonging to this week only
    week_values = df_ttt[(df_ttt['trade_open'].dt.date >= start_of_week) & (df_ttt['trade_open'].dt.date <= end_of_week)]
    week_values
    profit_value=week_values['profit'].max()
    loss_value=week_values['profit'].min()
    lowest_profit_row = week_values.loc[week_values['profit'].idxmin()]
    lowest_profit_datevalue=lowest_profit_row['trade_open']


    highest_profit_row = week_values.loc[week_values['profit'].idxmax()]
    highestprofit_datevalue=highest_profit_row['trade_open']
    week_sell_values = week_values[week_values['trade_type'] == 'Sell']
    week_sell_profit = week_sell_values['profit'].sum()
    weeks_total_sells=len(week_sell_values)

    week_buy_values = week_values[week_values['trade_type'] == 'Buy']
    week_buy_profit = week_buy_values['profit'].sum()
    weeks_total_buys=len(week_buy_values)
    week_profitablebuys = week_values[(week_values['trade_type'] == 'Buy') & (week_values['profit'] > 0)]
    week_buyprofit = week_buy_values['profit'].sum()
    weeks_total_profitable_buys = len(week_buy_values)

    week_profitablesells = week_values[(week_values['trade_type'] == 'Sell') & (week_values['profit'] > 0)]
    week_sellprofit = week_sell_values['profit'].sum()
    weeks_total_profitable_sells = len(week_sell_values)

    week_buy_loss=week_values[(week_values['trade_type'] == 'Buy') & (week_values['profit'] < 0)]
    week_buy_total_loss=week_buy_loss['profit'].sum()
    week_totalbuy_loss=len(week_buy_loss)

    week_sell_loss=week_values[(week_values['trade_type'] == 'Sell') & (week_values['profit'] < 0)]
    
    week_sell_total_loss=week_sell_loss['profit'].sum()
    week_totalsell_loss=len(week_sell_loss)

    return profit_value,loss_value,lowest_profit_datevalue,highestprofit_datevalue,week_sell_profit,weeks_total_sells,week_buy_profit,weeks_total_buys,week_buyprofit,weeks_total_profitable_buys,week_sellprofit,weeks_total_profitable_sells,week_buy_total_loss,week_totalbuy_loss,week_sell_total_loss,week_totalsell_loss



In [ ]:
def get_monthly_data():

# Get the current month's start and end dates
    today = datetime.date.today()
    start_of_month = datetime.date(today.year, today.month, 1)
    end_of_month = datetime.date(today.year, today.month + 1, 1) - datetime.timedelta(days=1)

    # Filter the DataFrame to include values belonging to this month only
    this_month_values = df_ttt[(df_ttt['trade_open'].dt.date >= start_of_month) & (df_ttt['trade_open'].dt.date <= end_of_month)]

    month_profit_value=this_month_values['profit'].max()
    month_loss_value=this_month_values['profit'].min()

    month_sell_values = this_month_values[this_month_values['trade_type'] == 'Sell']
    month_sell_profit = month_sell_values['profit'].sum()
    months_total_sells=len(month_sell_values)

    month_buy_values = this_month_values[this_month_values['trade_type'] == 'Buy']
    month_buy_profit = month_buy_values['profit'].sum()
    months_total_buys=len(month_buy_values)

    month_profitablebuys = this_month_values[(this_month_values['trade_type'] == 'Buy') & (this_month_values['profit'] > 0)]
    month_buyprofit = month_buy_values['profit'].sum()
    months_total_profitable_buys = len(month_buy_values)

    month_profitablesells = this_month_values[(this_month_values['trade_type'] == 'Sell') & (this_month_values['profit'] > 0)]
    month_sellprofit = month_sell_values['profit'].sum()
    months_total_profitable_sells = len(month_sell_values)

    month_buy_loss=this_month_values[(this_month_values['trade_type'] == 'Buy') & (this_month_values['profit'] < 0)]
    month_buy_total_loss=month_buy_loss['profit'].sum()
    month_totalbuy_loss=len(month_buy_loss)
    
    monthly_sell_loss=this_month_values[(this_month_values['trade_type'] == 'Sell') & (this_month_values['profit'] < 0)]
    month_sell_total_loss=monthly_sell_loss['profit'].sum()
    month_totalsell_loss=len(monthly_sell_loss)
    
    
    return month_profit_value,month_loss_value,month_sell_profit,months_total_sells,month_buy_profit,months_total_buys,month_buy_total_loss,month_totalbuy_loss,month_sell_total_loss,month_totalsell_loss,month_buyprofit,months_total_profitable_buys,month_sellprofit,months_total_profitable_sells
